# Sensitivities computation via automated adjoint for a 2D shallow water model

This notebooks seeks to demonstrate how to compute sensitivities of a cost function with respect to control parameters using automated adjoints, which is a tool provided by the Firedrake library. The model used in this notebook is a 2D shallow water model on a sphere. 

## Cost function and model
The cost function measures the summation of the kinetic and the potential energy of the fluid, which is modelled by the following expression:
\begin{equation}
J = \int_{\Omega} \left( \frac{1}{2} \textbf{u} \cdot \textbf{u} + \frac{1}{2} g D^2 \right) \, dx,
\end{equation}
where $\textbf{u}$ is the velocity of the fluid, $D$ is the fluid depth, and $g$ is the gravitational acceleration. The model is governed by the following set of equations:

\begin{align}
    \textbf{u}_t + (\textbf{u} \cdot \nabla) \textbf{u} + f \textbf{u}^{\perp} + g \nabla (D + b)  &= 0, \tag{2}\\
    D_t + \nabla \cdot (D \textbf{u}) &= 0, \tag{3}
\end{align}
where $f$ is the Coriolis parameter, the fluid depth is given by $D = H+h-b$, where $H$ is the mean fluid depth, $h$ is the free surface height and $b$ is the topography. No boundary conditions are required as in this case we are solving on a spherical domain.

## Solve the model with Gusto
We first import the libraries required for the computations using Gusto, which is a library built on top of Firedrake.

In [1]:
from firedrake import *
from gusto import *

INFO     Running /Users/ddolci/work/firedrake/lib/python3.13/site-packages/ipykernel_launcher.py --f=/Users/ddolci/Library/Jupyter/runtime/kernel-v38f09c6c2de87f5b866ca966324397dff906401c5.json


We then define the shallow water parameter $H$ (The fluid depth).

In [2]:
H = 5960.0
parameters = ShallowWaterParameters(H=H)

We will use one of the spherical meshes provided by Firedrake: the ``IcosahedralSphereMesh``. As the spherical domain we are solving over is the Earth we specify the radius as 6371220m. The refinement level, ``ref_level``, specifies the number of times the base icosahedron is refined. The argument ``degree`` specifies the polynomial degree of the function space used to represent the coordinates.

In [3]:
R = 6371220. # Earth radius in meters
mesh = IcosahedralSphereMesh(radius=R, refinement_level=3, degree=2)

A domain is created with Gusto ``Domain`` object, which holds the mesh and the function spaces defined on it.
The Domain object that is provided in this module contains the model’s mesh and the set of compatible function spaces defined upon it. It also holds the model’s time interval. 

In [4]:
dt = 900.
domain = Domain(mesh, dt, 'BDM', 1)

We can now set up the finite element form of the shallow water equations by passing ``ShallowWaterEquations`` Gusto class the following arguments: ``domain``, ``parameters``,  the expressions ``fexpr`` for the Coriolis parameter and `bexpr` for the bottom surface of the fluid.

In [5]:
x = SpatialCoordinate(mesh)
Omega = parameters.Omega  # Spatial domain
fexpr = 2*Omega*x[2]/R  # Expression for the Coriolis parameter
# ``lonlatr_from_xyz`` is returning the required spherical longitude ``lamda``, latitude ``theta``.
lamda, theta, _ = lonlatr_from_xyz(x[0], x[1], x[2])
R0 = pi/9. # radius of mountain (rad)
R0sq = R0**2  
lamda_c = -pi/2. # longitudinal centre of mountain (rad)
lsq = (lamda - lamda_c)**2 
theta_c = pi/6.  # latitudinal centre of mountain (rad)
thsq = (theta - theta_c)**2
rsq = min_value(R0sq, lsq+thsq)
r = sqrt(rsq)
bexpr = 2000 * (1 - r/R0)  # An expression for the bottom surface of the fluid.
eqn = ShallowWaterEquations(domain, parameters, fexpr=fexpr, bexpr=bexpr)

We specify instructions regarding the output using the `OutputParameters` class. The directory name, `dirname`, must be specified. To prevent losing hard-earned simulation data, Gusto will not allow existing files to be overwritten. Hence, if one wishes to re-run a simulation with the same output filename, the existing results file needs to be moved or deleted first. This is also the place to specify the output frequency (in timesteps) of vtu files. The default is `dumpfreq=1` which outputs vtu files at every timestep (very useful when first setting up a problem!). Below we set `dumpfreq=5`.

We can specify which diagnostics to record over a simulation. The list of avaliable diagnostics can be found in the gusto source code: [diagnostics](https://github.com/firedrakeproject/gusto/blob/main/gusto/diagnostics/diagnostics.py). Since this flow should be in a steady state, it is also instructive to output the steady state error fields for both $\textbf{u}$ and $D$ as this will tell us how close the simulation is to being correct. They will not be identically zero, due to numerical discretisation error, but the errors should not grow in time and they should reduce as the mesh and timestep are refined. We pass these diagnostics into the `IO` class, which controls the input and output and stores the fields which will be updated at each timestep.

In [6]:
tmpdir = 'outflow'
output = OutputParameters(dirname=str(tmpdir), log_courant=False)
io = IO(domain, output)

We now start to build a setup for time discretization. We use here the ``SemiImplicitQuasiNewton`` approach, which splits the equation into
`transport` terms and 'forcing' terms (i.e. everything that's not transport) and solves each separately. This allows for different
time-steppers to be used for transporting the velocity and depth fields. We choose to use an Implicit Midpoint method for the velocity
and an explicit strong stability preserving RK3 (SSPRK3) method for the depth. Since the Courant number for a stable SSPRK3 scheme
is lower than that for the Implicit Midpoint method, we do 2 subcycles of the SSPRK3 scheme per timestep, allowing us to use a longer timestep
overall. A full list of avaliable time stepping methods can be found at: [time discretisation](https://github.com/firedrakeproject/gusto/blob/main/gusto/time_discretisation.py).

In [7]:
# Transport schemes
transported_fields = [TrapeziumRule(domain, "u"), SSPRK3(domain, "D")]
transport_methods = [DGUpwind(eqn, "u"), DGUpwind(eqn, "D")]

# Time stepper
stepper = SemiImplicitQuasiNewton(eqn, io, transported_fields, transport_methods)

INFO     Physical parameters that take non-default values:
INFO     H: 5960.0


We are now ready to specify the initial conditions:
\begin{align}
    \textbf{u}_0 &= \frac{u_{max}}{R} [-y,x,0], \tag{4}\\
    D_0 &= H - \frac{\Omega u_{max} z^2}{g R} \tag{5}
\end{align}
Due to our choice of function spaces for the velocity and depth, the initialisations of each variable use projection and interpolation operations respectively.

In [8]:
u0 = stepper.fields('u')
D0 = stepper.fields('D')
u_max = 20.   # Maximum amplitude of the zonal wind (m/s)
uexpr = as_vector([-u_max*x[1]/R, u_max*x[0]/R, 0.0])
g = parameters.g  # acceleration due to gravity (m/s^2)
Rsq = R**2
Dexpr = H - ((R * Omega * u_max + 0.5*u_max**2)*x[2]**2/Rsq)/g - bexpr

u0.project(uexpr)
D0.interpolate(Dexpr)

Coefficient(WithGeometry(IndexedProxyFunctionSpace(<firedrake.mesh.MeshTopology object at 0x16ab42ba0>, FiniteElement('Discontinuous Lagrange', triangle, 1), name='L2', index=1, component=None), Mesh(VectorElement(FiniteElement('Lagrange', triangle, 2), dim=3), 4)), 90)

When using the `SemiImplicitQuasiNewton` timestepper, we also have to set up any non-zero reference profiles.

In [9]:
Dbar = Function(D0.function_space()).assign(H)
stepper.set_reference_profiles([('D', Dbar)])

We are almost read to run the model. Before we do so, we have to tape the model in ordem to compute the sensititivities via automated adjoint. Henve, at this point we import ``firedrake.adjoint`` and enable the tape of the forward solver (Shallow Water model) with ``continue_annotation()``.

In [10]:
from firedrake.adjoint import *
continue_annotation()

True

We are finally ready to run the model!

In [11]:
timesteps = 5 
stepper.run(0., timesteps*dt)

INFO     
INFO     ========================================
INFO     at start of timestep 1, t=0.0, dt=900.0
INFO     Semi-implicit Quasi Newton: Explicit forcing
INFO     Semi-implicit Quasi Newton: Transport 0: u
INFO     Semi-implicit Quasi Newton: Transport 0: D
INFO     Semi-implicit Quasi Newton: Implicit forcing (0, 0)
INFO     Semi-implicit Quasi Newton: Mixed solve (0, 0)
INFO     Semi-implicit Quasi Newton: Implicit forcing (0, 1)
INFO     Semi-implicit Quasi Newton: Mixed solve (0, 1)
INFO     Semi-implicit Quasi Newton: Transport 1: u
INFO     Semi-implicit Quasi Newton: Transport 1: D
INFO     Semi-implicit Quasi Newton: Implicit forcing (1, 0)
INFO     Semi-implicit Quasi Newton: Mixed solve (1, 0)
INFO     Semi-implicit Quasi Newton: Implicit forcing (1, 1)
INFO     Semi-implicit Quasi Newton: Mixed solve (1, 1)
INFO     
INFO     ========================================
INFO     at start of timestep 2, t=900.0, dt=900.0
INFO     Semi-implicit Quasi Newton: Explicit forc

We then compute the final solution of the shallow water solver that is used to compute the cost functional (1) as shown below.

In [12]:
u = stepper.fields('u')  # Final velocity field
D = stepper.fields('D')  # Final depth field

J = assemble(0.5*inner(u, u)*dx + 0.5*g*D**2*dx)

We can finally run our sensitivity computation: gradient of the cost function with respect initial fluid depth. The computation of the gradient can be done using the `compute_gradient` function from the `firedrake.adjoint` module. Before, we have to define the control variable.

In [13]:
control = [Control(D0), Control(u0)]
grad = compute_gradient(J, control)

We can also verify the gradient computation with second order [Taylor's test](https://www.dolfin-adjoint.org/en/latest/documentation/verification.html). To do so, we define the reduced functional as follows:

In [14]:
J_hat = ReducedFunctional(J, control)

We then verify the gradient computation using the `taylor_test` function from the `firedrake.adjoint` module.

In [ ]:
h0 = Function(D0.function_space()).interpolate(1.)
h1 = Function(u0.function_space()).assign(1.)
taylor_test(J_hat, [D0, u0], [h0, h1])

ValueError: 1 perturbations are given but only 2 expansion points are provided

That is great! He have a senitivity computation via automated adjoint with Gusto! In addition, we have verified the gradient computation with a second order Taylor's test, which is a good practice to ensure the correctness of the gradient computation. See the results above are close to the expected values, 2.0.

For a notebook demosntration of adjoints, it is a good practice clear the tape, and stop annotating the forward solver.

In [ ]:
# tape = get_working_tape()
# tape.clear_tape()